<h1><center>Классификация текстов</center></h1>

Сегодня мы подробнее познакомимся с задачей классификации текстов на два или более классов.

У нас есть [датасет отзывов на фильмы IMDB](https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews), который часто используют при разработке и оценке моделей классификации текстов. Каждый из отзывов в датасете имеет свою оценку: является ли он позитивным или негативным.

**Цель:** Используя данные отзывов IMDB, построить модель, которая на основе текста отзыва сможет определять, относится ли он к позитивнму или к негативному классу.

## План работы
* Подготовка данных
* Классификация с помощью Word2Vec:
    * Обучение модели Word2Vec
    * Построение векторов для текстов через среднее векторов слов
    * Построение векторов для текстов через средневзвешенное векторов слов
* Классификация с помощью FastText
* Классификация с помощью сверточной нейронной сети (CNN)

## Подготовка данных



Для обучения нейросетевых моделей (далее) мы будем использовать популярный фреймворк [PyTorch](https://pytorch.org/).

В предположении, что PyTorch уже установлен, поставим дополнительные модули (torchvision и torchtext) и загрузим модель для токенизации:

In [11]:
!pip install torchvision

In [12]:
!pip install torchtext

Загрузим датасет IMDB из torchtext, это будет удобно для дальнейшего использования этих данных при обучении нейронной сети.

In [15]:
import numpy as np
import torch
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score,classification_report,confusion_matrix
from torchtext.legacy import data
from torchtext.legacy import datasets
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter,defaultdict
%matplotlib inline
import seaborn as sns

print(torch.__version__)

SEED = 0
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField()

train_src, test = datasets.IMDB.splits(TEXT, LABEL)

1.9.0+cu102


Разделим данные на обучаующую и тестовую выборку и преобразуем их в формат, удобный для построения классических моделей машинного обучения.

In [16]:
X_train = [example.text for example in train_src]
y_train = [example.label for example in train_src]

X_test = [example.text for example in test]
y_test = [example.label for example in test]


In [17]:
from collections import Counter

print ("total train examples %s" % len(y_train))
print ("total test examples %s" % len(y_test))

train_counter = Counter(y_train)
test_counter = Counter(y_test)

print ("{0} positive and {1} negative examples in test".format(test_counter['pos'],test_counter['neg']))
print ("{0} positive and {1} negative examples in test".format(test_counter['pos'],test_counter['neg']))

total train examples 25000
total test examples 25000
12500 positive and 12500 negative examples in test
12500 positive and 12500 negative examples in test


Можно видеть, что классы сбалансированы: количество позитивных и негативных отзывов совпадают.

Посмотрим на примеры позитивного и негативного отзыва.

In [18]:
print("Negative example:\n{0}\n\n".format(' '.join(X_train[0])))
print("Positive example:\n{0}\n\n".format(' '.join(X_train[-1])))

Negative example:
This film has been scaring me since the first day I saw it.<br /><br />My Mum had watched it when it was on the telly back in ' 92 . I remember being woken up in the middle of the night by her tearful ramblings as my Dad helped her up the stairs.<br /><br />She was saying something like " Do n't let her get me " or something like that . I asked what had made her so upset and she told me that she 'd been watching The Woman in Black.<br /><br />So obviously i had to watch it and even though i was only eleven she let me . It scared the s * * * out of me . I 've been immune to horror films since watching this !


Positive example:
Repugnant Bronson thriller . Unfortunately , it 's technically good and I gave it 4/10 , but it 's so utterly vile that it would be inconceivable to call it " entertainment " . Far more disturbing than a typical slasher film .




## Классификация с помощью Word2Vec

Подробно языковая модель *Word2Vec* была рассмотрена в предыдущем семинаре.

Здесь мы будем использовать эту модель для получения векторных представлений слов.

In [19]:
from gensim.models import Word2Vec
model = Word2Vec(X_train, size=100, window=5, min_count=5, workers=4)
model.save("word2vec.model")

KeyboardInterrupt: ignored

In [ ]:
w2v_model = Word2Vec.load("word2vec.model")

Чтобы решить задачу классификации текстов, нам нужно каждому тексту поставить в соответствие вектор.

Реализуем класс *MeanEmbeddingVectorizer*, с помощью которого будем получать вектора для текстов отзывов на основе векторов слов.
Вектор каждого текста будет равен среднему векторов входящих в него слов.

In [ ]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = len(list(word2vec.values())[0])

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

Попробуем также немного другой подход $-$ будем получать вектор для текста как средневзвешенное векторов входящих в него слов с весами *tf-idf*.

Для этого реализуем класс *TfidfEmbeddingVectorizer*.

In [ ]:
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = len(list(word2vec.values())[0])

    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

Преобразуем модель Word2Vec в словарь для дальнейшего использования для векторизации текстов.

In [ ]:
w2v = dict(zip(w2v_model.wv.index2word, w2v_model.wv.syn0))

Для классификации будем использовать модель [случайного леса](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) из scikit-learn.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

rfc_w2v = Pipeline([
    ("word2vec vectorizer", MeanEmbeddingVectorizer(w2v)),
    ("extra trees", RandomForestClassifier(n_estimators=20))])
rfc_w2v_tfidf = Pipeline([
    ("word2vec vectorizer", TfidfEmbeddingVectorizer(w2v)),
    ("extra trees", RandomForestClassifier(n_estimators=20))])

In [ ]:
%time rfc_w2v.fit(X_train,y_train)
pred = rfc_w2v.predict(X_test)

Построим отчет по предсказаниям модели (с помощью MeanEmbeddingVectorizer) на тестовой выборке. Выведем точность (precision), полноту (recall) и f1-меру для позитивного и негативного классов, а также количество объектов каждого класса (support) в тестовой выборке.

In [ ]:
print("Precision: {0:6.2f}".format(precision_score(y_test, pred, average='macro')))
print("Recall: {0:6.2f}".format(recall_score(y_test, pred, average='macro')))
print("F1-measure: {0:6.2f}".format(f1_score(y_test, pred, average='macro')))
print("Accuracy: {0:6.2f}".format(accuracy_score(y_test, pred)))
print(classification_report(y_test, pred))
labels = rfc_w2v.classes_


sns.heatmap(data=confusion_matrix(y_test, pred), annot=True, fmt="d", cbar=False, xticklabels=labels, yticklabels=labels)
plt.title("Confusion matrix")
plt.show()

В данном случае классы сбалансированы (количество объектов позитивного и негативного классов совпадают), поэтому в качестве метрики качества можно использовать *accuracy*.

In [ ]:
%time rfc_w2v_tfidf.fit(X_train,y_train)
pred = rfc_w2v_tfidf.predict(X_test)

Теперь выведем результаты работы классификатора на признаках из *TfidfEmbeddingVectorizer*.

In [ ]:
print("Precision: {0:6.2f}".format(precision_score(y_test, pred, average='macro')))
print("Recall: {0:6.2f}".format(recall_score(y_test, pred, average='macro')))
print("F1-measure: {0:6.2f}".format(f1_score(y_test, pred, average='macro')))
print("Accuracy: {0:6.2f}".format(accuracy_score(y_test, pred)))
print(classification_report(y_test, pred))
labels = rfc_w2v.classes_


sns.heatmap(data=confusion_matrix(y_test, pred), annot=True, fmt="d", cbar=False, xticklabels=labels, yticklabels=labels)
plt.title("Confusion matrix")
plt.show()

Можно видеть, что при использовании *TfidfEmbeddingVectorizer* метрика Accuracy немного увеличилась по сравнению c *MeanEmbeddingVectorizer*.

## Классификация с помощью FastText

Теперь попробуем классифицировать тексты с помощью другой языковой модели [FastText](https://github.com/facebookresearch/fasttext), которая также была описана в предыдущем занятии.

Чтобы использовать модель, подготовим для нее данные, записав тексты и их лейблы классов в файлы data.train.txt и text.txt

In [ ]:
!pip install fasttext

In [ ]:
import fasttext

N = -1

with open('data.train.txt', 'w+') as outfile:
    for i in range(len(X_train)):
        outfile.write('__label__' + y_train[i] + ' '+ ' '.join(X_train[i]) + '\n')
    

with open('test.txt', 'w+') as outfile:
    for i in range(len(X_test)):
        outfile.write('__label__' + y_test[i] + ' ' + ' '.join(X_test[i]) + '\n')

Будем использовать встроенную в fasttext модель классификации с помощью метода train_supervised.

In [ ]:
classifier = fasttext.train_supervised('data.train.txt')#, 'model')
result = classifier.test('test.txt')
print('P@1:', result[1])#.precision)
print('R@1:', result[2])#.recall)
print('Number of examples:', result[0])#.nexamples)

In [ ]:
pred = classifier.predict([' '.join(x) for x in X_test])[0]
pred = [label[0].split("_")[-1] for label in pred]

print("Precision: {0:6.2f}".format(precision_score(y_test, pred, average='macro')))
print("Recall: {0:6.2f}".format(recall_score(y_test, pred, average='macro')))
print("F1-measure: {0:6.2f}".format(f1_score(y_test, pred, average='macro')))
print("Accuracy: {0:6.2f}".format(accuracy_score(y_test, pred)))
print(classification_report(y_test, pred))#[i[0] for i in pred]))
labels = rfc_w2v.classes_


sns.heatmap(data=confusion_matrix(y_test, pred), annot=True, fmt="d", cbar=False, xticklabels=labels, yticklabels=labels)
plt.title("Confusion matrix")
plt.show()

## Классификация с использованием CNN

Теперь попробуем применить нейросетевой подход.

Построим сверточную нейронную сеть (CNN) для классификации документов на позитивный и негативный классы.

In [20]:
!pip install spacy

In [21]:
!python -m spacy download en
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 12.0 MB 5.3 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
     |████████████████████████████████| 12.0 MB 5.1 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [22]:
import spacy

en_nlp = spacy.load('en_core_web_sm')

Попробуем обучить простую CNN на векторах слов. С учётом того, что в коллекции 100К уникальных слов, и векторы получатся достаточно громоздкие, урежем коллекцию до 25К слов, для всех прочих заведя токен unk (unknown). Кроме того, разобьём обучающий сет на обучение и валидацию для настройки параметров.

In [23]:
import random
train, valid = train_src.split(random_state=random.seed(SEED))

In [24]:
TEXT.build_vocab(train, max_size=25000)
LABEL.build_vocab(train)

Выше в словаре, помимо отсечения 25К наиболее частых слов, также появятся два специальных токена - unk и pad (padding).

Теперь создадим батчи, предварительно отсортировав тексты по длине, что минимизировать вставки pad-токенов:

In [25]:
BATCH_SIZE = 16

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, valid, test), 
    batch_size=BATCH_SIZE, 
    sort_key=lambda x: len(x.text), 
    repeat=False)

Опишем функцию подсчёта accuracy, а также функции обучения и применения сети:

In [26]:
import torch.nn.functional as F

def binary_accuracy(preds, y):
    rounded_preds = torch.round(F.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

In [27]:
def train_func(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        
        predictions = model(batch.text.cuda()).squeeze(1)

        loss = criterion(predictions.float(), batch.label.float().cuda())
        acc = binary_accuracy(predictions.float(), batch.label.float().cuda())
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss
        epoch_acc += acc
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [28]:
def evaluate_func(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text.cuda()).squeeze(1)

            loss = criterion(predictions.float(), batch.label.float().cuda())
            acc = binary_accuracy(predictions.float(), batch.label.float().cuda())

            epoch_loss += loss
            epoch_acc += acc
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Начнём с подготовки данных:

In [29]:
TEXT.build_vocab(train, max_size=25000, vectors="glove.6B.100d")
LABEL.build_vocab(train)


BATCH_SIZEBATCH_S  = 8

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, valid, test), 
    batch_size=BATCH_SIZE, 
    sort_key=lambda x: len(x.text), 
    repeat=False)

.vector_cache/glove.6B.zip: 862MB [02:56, 4.88MB/s]                           
100%|█████████▉| 399999/400000 [00:23<00:00, 16763.25it/s]


Для создания свёрточного слоя воспользуемся nn.Conv2d, in_channels в нашем случае один (текст), out_channels -- это число число фильтров и размер ядер всех фильтров. Каждый фильтр будет иметь размерность [n x размерность эмбеддинга], где n - размер обрабатываемой n-граммы.

Важно, что предложения имели длину не меньше размера самого большого из используемых фильтров (здесь это не страшно, поскольку в используемых данных нет текстов, состоящих из пяти и менее слов).

In [30]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv_0 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[0], embedding_dim))
        self.conv_1 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[1], embedding_dim))
        self.conv_2 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[2], embedding_dim))
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        #x = [sent len, batch size]
        x = x.permute(1, 0)
                
        #x = [batch size, sent len]
        embedded = self.embedding(x)
                
        #embedded = [batch size, sent len, emb dim]
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        conved_0 = F.relu(self.conv_0(embedded).squeeze(3))
        conved_1 = F.relu(self.conv_1(embedded).squeeze(3))
        conved_2 = F.relu(self.conv_2(embedded).squeeze(3))
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
        
        #pooled_n = [batch size, n_filters]
        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2), dim=1))

        #cat = [batch size, n_filters * len(filter_sizes)]
        return self.fc(cat)

Cоздадим модель и загрузим предобученные эмбеддинги:


In [31]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT)

In [32]:
TEXT.build_vocab(train, max_size=25000, vectors="glove.6B.100d")
LABEL.build_vocab(train)

pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [33]:
BATCH_SIZE  = 64

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, valid, test), 
    batch_size=BATCH_SIZE, 
    sort_key=lambda x: len(x.text), 
    repeat=False)

Используя определённые ранее функции, запустим обучение с оптимизатором Adam и оценим качество на валидации и тесте:

In [34]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

model = model.cuda()

In [35]:
N_EPOCHS = 5

for epoch in range(N_EPOCHS):
    train_loss, train_acc = train_func(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate_func(model, valid_iterator, criterion)
    
    print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc*100:.2f}%, Val. Loss: {valid_loss:.3f}, Val. Acc: {valid_acc*100:.2f}%')

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: 01, Train Loss: 0.498, Train Acc: 74.35%, Val. Loss: 0.346, Val. Acc: 85.32%
Epoch: 02, Train Loss: 0.302, Train Acc: 87.41%, Val. Loss: 0.297, Val. Acc: 87.42%
Epoch: 03, Train Loss: 0.217, Train Acc: 91.38%, Val. Loss: 0.280, Val. Acc: 88.78%
Epoch: 04, Train Loss: 0.147, Train Acc: 94.71%, Val. Loss: 0.280, Val. Acc: 89.12%
Epoch: 05, Train Loss: 0.086, Train Acc: 97.22%, Val. Loss: 0.307, Val. Acc: 88.78%


In [36]:
test_loss , test_acc = evaluate_func(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f}, Test Acc: {test_acc*100:.2f}%')

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Test Loss: 0.311, Test Acc: 88.36%



Можно видеть, что качество классификации после 5 эпох обучения стало выше, чем при использовании FastText.

## Итоги

На этом занятии мы рассмотрели задачу бинарной классификации текстов на примере отзывов IMDB.
Для решения этой задачи мы попробовали следующие модели:
1. Word2Vec + MeanEmbeddingVectorizer + RandomForest
2. Word2Vec + TfidfEmbediingVectorizer + RandomForest
3. FastText со встроенным классификатором
4. Glove + CNN

Наилучшее качество показала нейросетевая модель CNN.